In [9]:
##灰色预测 参考https://blog.csdn.net/qq_29831163/article/details/89714074
import numpy as np                  # 导入模块 numpy，并简写成 np
import math

#级比检验以及数据偏移
def TestData(X,nVar):
    Y = np.zeros((nVar)) 
    a = math.exp(-2/(nVar+1))    #级比检验下界
    b = math.exp(2/(nVar+1))     #级比检验上界
    minx = X[0]
    flag = 0
    c = 0                        #偏移常数
    for i in range(nVar):
        if minx > X[i]:
            minx = X[i]
        if X[i] == 0:
            flag = 1
    if flag == 1 or minx < 0:
        c = abs(minx)+1
        for i in range(nVar):
            X[i] += c
    times = 0
    while times < 10000:
        flag = 0
        times += 1
        for i in range(1,nVar):
            Y[i] = X[i-1]/X[i]
            if Y[i] < a or Y[i] > b:
                flag = 1
                break
        if flag == 0:
            break
        else:
            c += 1
            for i in range(nVar):
                X[i] += 1
    return X,Y,c          #X：偏移后的数列，Y：级比检验数列，c：偏移常数

#建立GM(1,1)模型，求解灰微分方程参数a、b
def GM1_1(X_c,nVar):
    B = np.zeros((nVar-1,2))
    Y = np.zeros((nVar-1))
    u = np.zeros((2,1)) 
    for i in range(nVar-1):
        Y[i] = X_c[i+1]
        B[i][1] = 1
    for i in range(1,nVar):
        X_c[i] += X_c[i-1]
        B[i-1][0] = -(X_c[i]+X_c[i-1])/2
#     print(X_c)
    BT = B.transpose()
    u = np.matmul(np.matmul(np.linalg.inv(np.matmul(BT,B)),BT),Y)
    a = u[0]
    b = u[1]
    return a,b

#分析数据，检测模型准确度
def Analysis(X,X_c,Y,c,a,b,nVar,forenum): #X：原数列，X_c:偏移数列,Y:级比数列
    X1 = np.zeros((nVar+forenum)) 
    Y1 = np.zeros((nVar))  #残差
    Y2 = np.zeros((nVar))  #相对误差
    Y3 = np.zeros((nVar))  #级比偏差
    d = X_c[0]-b/a
    e = b/a
    for i in range(nVar-1+forenum):
        X1[i+1] = d*math.exp(-a*(i+1))+e
    X1[0] = X_c[0]
    for i in range(nVar-1+forenum,0,-1):
        X1[i] -= X1[i-1]+c 
    X1[0] -= c
    for i in range(nVar):
        Y1[i] = X[i]-X1[i]
        Y2[i] = abs(X[i]-X1[i])/X[i]
        Y3[i] = 1-((1-0.5*a)/(1+0.5*a))*Y[i]
    return X1,Y1,Y2,Y3,X1[nVar:]

def grey(X,nVar,forenum):
    X_c = np.zeros((nVar)) 
    [X_c,Y,c] = TestData(X,nVar)
    [a,b] = GM1_1(X_c,nVar)
    [X1,Y1,Y2,Y3,fore] = Analysis(X,X_c,Y,c,a,b,7,3)
    return fore


def main():
    X = [71.1,72.4,72.4,72.1,71.4,72.0,71.6]
#     X_c = np.zeros((7)) 
#     [X_c,Y,c] = TestData(X,7)
#     [a,b] = GM1_1(X_c,7)
#     [X1,Y1,Y2,Y3,fore] = Analysis(X,X_c,Y,c,a,b,7,3)
# #     print(a,b)
#     print(X1)
# #     print(Y1)
# #     print(Y2)
# #     print(Y3)
    fore = grey(X,len(X),3)
    print(fore)
if __name__ == '__main__':
    main()
        

[71.39464589 71.22750803 71.06076145]
